In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, KBinsDiscretizer
from sklearn.compose import TransformedTargetRegressor
import pandas as pd
import tensorflow as tf
import numpy as np
import json

#  Import and read the charity_data.csv.
import pandas as pd 

In [2]:
initial_df = pd.read_json("drifting_ll_1.json")
initial_df.head()

,mmsi,timestamp,speed,course,lat,lon,is_fishing
0,12639560807591,2012-01-21 09:01:44,8.2,230.500000,14.865583,-26.853662,-1
1,12639560807591,2012-01-21 09:03:25,7.3,238.399994,14.863870,-26.856800,-1
2,12639560807591,2012-01-21 09:05:34,6.8,238.899994,14.861551,-26.860649,-1
3,12639560807591,2012-01-21 10:54:41,6.9,251.800003,14.822686,-26.865898,-1
4,12639560807591,2012-01-21 10:55:41,6.1,231.100006,14.821825,-26.867579,-1


In [7]:
initial_df['lat'] = initial_df['lat'].astype(int)
initial_df['lon'] = initial_df['lon'].astype(int)

In [25]:
#mmsi is an identifier (aka the boat name). drop it
# time stamp? 
# what does the course number mean 
# -1=standard 'loitering', other two are more active? 1 0
# round lat and lon to ints for sanity's sake
# process timestamp into year month? 
month_list = []
for i in range(len(initial_df['timestamp'])):
    month_list.append(initial_df['timestamp'][i].month_name())

initial_df["Month"] = month_list
initial_df.head()

,mmsi,timestamp,speed,course,lat,lon,is_fishing,Month
0,12639560807591,2012-01-21 09:01:44,8.2,230.500000,14,-26,-1,January
1,12639560807591,2012-01-21 09:03:25,7.3,238.399994,14,-26,-1,January
2,12639560807591,2012-01-21 09:05:34,6.8,238.899994,14,-26,-1,January
3,12639560807591,2012-01-21 10:54:41,6.9,251.800003,14,-26,-1,January
4,12639560807591,2012-01-21 10:55:41,6.1,231.100006,14,-26,-1,January


In [27]:
year_list = []
for i in range(len(initial_df['timestamp'])):
    year_list.append(initial_df['timestamp'][i].year)

initial_df["Year"] = year_list
initial_df.head()

,mmsi,timestamp,speed,course,lat,lon,is_fishing,Month,Year
0,12639560807591,2012-01-21 09:01:44,8.2,230.500000,14,-26,-1,January,2012
1,12639560807591,2012-01-21 09:03:25,7.3,238.399994,14,-26,-1,January,2012
2,12639560807591,2012-01-21 09:05:34,6.8,238.899994,14,-26,-1,January,2012
3,12639560807591,2012-01-21 10:54:41,6.9,251.800003,14,-26,-1,January,2012
4,12639560807591,2012-01-21 10:55:41,6.1,231.100006,14,-26,-1,January,2012


In [28]:
processing_df = initial_df.drop(["mmsi","timestamp"], axis=1)
processing_df.head()

,speed,course,lat,lon,is_fishing,Month,Year
0,8.2,230.500000,14,-26,-1,January,2012
1,7.3,238.399994,14,-26,-1,January,2012
2,6.8,238.899994,14,-26,-1,January,2012
3,6.9,251.800003,14,-26,-1,January,2012
4,6.1,231.100006,14,-26,-1,January,2012


In [29]:
to_test = pd.get_dummies(processing_df)
X = to_test.drop(['is_fishing'], axis=1)
y = to_test['is_fishing']
X.columns

Index(['speed', 'course', 'lat', 'lon', 'Year', 'Month_April', 'Month_August',
       'Month_December', 'Month_February', 'Month_January', 'Month_July',
       'Month_June', 'Month_March', 'Month_May', 'Month_November',
       'Month_October', 'Month_September'],
      dtype='object')

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')
training_score = clf.score(X_train_scaled, y_train)
testing_score = clf.score(X_test_scaled, y_test)

Training Score: 0.9996491228070176
Testing Score: 0.9733333333333334


In [34]:
clf.predict([[6.9, 230, 14, -8, 0,0,0,0,0,0,0,0,0,0,0,1, 2012]])

array([-1], dtype=int64)